# Fase 3 (Parte 2) – Correciones y Limpieza de los datos

**Importante** si bien se presenta código en este notebook es más bien para documentar algunos procesos que, como equipo, realizamos. Por lo que por medio de pipelines, ejecutando **Kedro Run**, se ejecuta toda la peparación y limpieza que se ve aquí junto a la unión de los Dataset's para posteriormente verlos en el DAG ejecutando, por consola de visual studio code, **Kedro viz**.

In [1]:
# Carga de contexto y librerias para el notebook :D
%load_ext kedro.ipython
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


////////////////AJUSTES PARA MODELOS DE CLASIFICACIÓN ///////////////////////////////////////////////////

Verificación inicial de las variables en 'PRI_FULL'

In [2]:
df = catalog.load("PRI_full")   # lee el dataset del catálogo
list(df.columns)  

[10/22/25 16:23:24] INFO     Loading data from PRI_full (ParquetDataset)...                    ]8;id=676340;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=297037;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\


[
    'TransactionID',
    'CustomerID',
    'CustLocation',
    'TransactionDate',
    'TransactionTime',
    'TransactionAmount (INR)',
    'txn_count',
    'total_spent',
    'avg_spent',
    'max_spent',
    'avg_balance',
    'first_txn_date',
    'last_txn_date',
    'recency_days',
    'tenure_days',
    'txn_per_day',
    'Recency',
    'Frequency',
    'Monetary',
    'RFM Score',
    'Segment_Final'
]

In [3]:
# =============================================
# 🔍 COMPARAR TRANSACCIONES ENTRE DOS DATASETS
# =============================================
import pandas as pd

# --- 1. Cargar ambos datasets ---
# Ajusta las rutas si están en otra carpeta
fraud_df = catalog.load("fraud_dataset")
pri_full_df = catalog.load("PRI_full")

# --- 2. Mostrar estructura básica ---
print("Dataset FRAUD:")
print("Shape:", fraud_df.shape)
print("Columnas:", fraud_df.columns.tolist(), "\n")

print("Dataset PRI_FULL:")
print("Shape:", pri_full_df.shape)
print("Columnas:", pri_full_df.columns.tolist(), "\n")

# --- 3. Identificar columnas comunes ---
common_cols = list(set(fraud_df.columns).intersection(set(pri_full_df.columns)))
print("✅ Columnas comunes encontradas:", common_cols, "\n")

# --- 4. Si existe TransactionID, comparar coincidencias ---
if "TransactionID" in common_cols:
    merged = pri_full_df.merge(fraud_df, on="TransactionID", how="inner")
    print(f"🔗 Transacciones en común: {len(merged)}")
    print(f"🔸 Total en FRAUD: {len(fraud_df)}")
    print(f"🔸 Total en PRI_FULL: {len(pri_full_df)}")

    # --- 5. Revisar si las del dataset FRAUD están incluidas en PRI_FULL ---
    if len(merged) == len(fraud_df):
        print("\n✅ El dataset FRAUD está completamente contenido en PRI_FULL (solo fraudes).")
    elif len(merged) == 0:
        print("\n⚠️ No se encontraron coincidencias. Verifica la columna TransactionID o el formato.")
    else:
        print("\n🟡 El dataset FRAUD comparte algunas transacciones con PRI_FULL, pero no todas.")

    # --- 6. (Opcional) Revisar proporción de fraudes ---
    if "is_fraud" in fraud_df.columns:
        fraud_ratio = fraud_df["is_fraud"].mean() * 100
        print(f"\n📊 Porcentaje de transacciones marcadas como fraude: {fraud_ratio:.2f}%")
else:
    print("⚠️ No hay columna 'TransactionID' común entre los datasets. Usa otra columna clave para comparar.")


[10/22/25 16:23:31] INFO     Loading data from fraud_dataset (CSVDataset)...                   ]8;id=94790;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=977890;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

                    INFO     Loading data from PRI_full (ParquetDataset)...                    ]8;id=59174;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=157778;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

Dataset FRAUD:
Shape: (9843, 22)
Columnas: ['Unnamed: 0', 'TransactionID', 'CustomerID', 'CustomerDOB', 'CustGender', 'CustLocation', 'CustAccountBalance', 'TransactionDate', 'TransactionTime', 'TransactionAmount (INR)', 'Age', 'AmountZScoreByLocation', 'DayOfWeek', 'IsWeekend', 'IsHoliday', 'TimeOfDay', 'IsLateNight', 'TimeSinceLastTxn', 'TxnCountInLast24Hours', 'IsAnomaly', 'RiskScore', 'Segment'] 

Dataset PRI_FULL:
Shape: (984240, 21)
Columnas: ['TransactionID', 'CustomerID', 'CustLocation', 'TransactionDate', 'TransactionTime', 'TransactionAmount (INR)', 'txn_count', 'total_spent', 'avg_spent', 'max_spent', 'avg_balance', 'first_txn_date', 'last_txn_date', 'recency_days', 'tenure_days', 'txn_per_day', 'Recency', 'Frequency', 'Monetary', 'RFM Score', 'Segment_Final'] 

✅ Columnas comunes encontradas: ['CustomerID', 'TransactionDate', 'TransactionID', 'TransactionAmount (INR)', 'CustLocation', 'TransactionTime'] 

🔗 Transacciones en común: 9843
🔸 Total en FRAUD: 9843
🔸 Total en PRI_

Como respuesta podemos ver que el dataset 'PRI_FULL' si contiene las transacciones fraudulentas, pero si revisamos el dataset 'fraud_dataset' podemos observar que tiene algunas variables de inter'es para el analisis.

In [4]:
df = catalog.load("fraud_dataset")   # lee el dataset del catálogo
list(df.columns)  

[10/22/25 16:23:38] INFO     Loading data from fraud_dataset (CSVDataset)...                   ]8;id=513922;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=118184;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\


[
    'Unnamed: 0',
    'TransactionID',
    'CustomerID',
    'CustomerDOB',
    'CustGender',
    'CustLocation',
    'CustAccountBalance',
    'TransactionDate',
    'TransactionTime',
    'TransactionAmount (INR)',
    'Age',
    'AmountZScoreByLocation',
    'DayOfWeek',
    'IsWeekend',
    'IsHoliday',
    'TimeOfDay',
    'IsLateNight',
    'TimeSinceLastTxn',
    'TxnCountInLast24Hours',
    'IsAnomaly',
    'RiskScore',
    'Segment'
]

Entre ellas podemos observar:
'IsAnomaly' - Esta variable nos dice si es fraudulenta o no ........
'IsWeekend' - Esta variable dice si es fin de semana
'IsHoliday' - Esta variable dice si es feriado, por el momento no podemos determinarla en el dataset porque es un dataset de otro país, descartada
'DayOfWeek' - Dia de la semana, también puede ser un punto de interés
'TimeOfDay' - A que momento del dia ocurre que también podría ser un aporte
'AmountZScoreByLocation' - Según los montos en zonas simkilares podría ser muy relevante, quizás en zonas determinadas el monto es inusual
'IsLateNight' - Otra variable de tiempo al igual que TimeOfDay podría ser un punto fuerte


A continuación intentamos replicar algunas de estas variables en el dataset principal 'PRI_FULL' para dar más información al momento de entrenar los modelos.

In [5]:
# === 1️⃣ Cargar dataset base ===
df = catalog.load("PRI_full").copy()

# --- Asegurar formato de fechas y horas ---
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')
df['TransactionTime'] = pd.to_datetime(df['TransactionTime'], format='%H:%M:%S', errors='coerce')

# =========================================================
# === 2️⃣ Variables temporales y de contexto ===============
# =========================================================

# Día de la semana
df['DayOfWeek'] = df['TransactionDate'].dt.day_name()

# Fin de semana
df['IsWeekend'] = df['DayOfWeek'].isin(['Saturday', 'Sunday']).astype(int)

# Hora del día
df['hour'] = df['TransactionTime'].dt.hour
df['IsLateNight'] = ((df['hour'] >= 23) | (df['hour'] <= 5)).astype(int)
df['TimeOfDay'] = pd.cut(
    df['hour'], bins=[0,6,12,18,24],
    labels=['Night','Morning','Afternoon','Evening'],
    right=False, include_lowest=True
)
# =========================================================
# === 3️⃣ Variables de comportamiento ======================
# =========================================================

# Monto Z-Score por ubicación
df['AmountZScoreByLocation'] = (
    df.groupby('CustLocation')['TransactionAmount (INR)']
      .transform(lambda x: (x - x.mean()) / (x.std() + 1e-9))
)

# Días desde última transacción por cliente
df = df.sort_values(['CustomerID', 'TransactionDate'])
df['TimeSinceLastTxn'] = (
    df.groupby('CustomerID')['TransactionDate']
      .diff().dt.total_seconds() / (60*60*24)
).fillna(0)

# --- Simplificación robusta ---
# Si la diferencia entre esta y la anterior <= 24h, marcamos como 1
df['TxnCountInLast24Hours'] = (
    (df.groupby('CustomerID')['TransactionDate']
       .diff().dt.total_seconds() / 3600 <= 24)
    .astype(int)
    .fillna(0)
)

# =========================================================
# === 4️⃣ Variables de riesgo ==============================
# =========================================================

# Riesgo basado en monto relativo al promedio histórico del cliente
df['RiskScore'] = np.where(
    df['TransactionAmount (INR)'] > (
        df['TransactionAmount (INR)'].mean() +
        2 * df['TransactionAmount (INR)'].std()
    ),
    1, 0
)

# Anomalía simple (monto alto + transacción nocturna)
df['IsAnomaly'] = ((df['RiskScore'] == 1) & (df['IsLateNight'] == 1)).astype(int)

# =========================================================
# === 5️⃣ Limpieza y validación ============================
# =========================================================
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna({
    'AmountZScoreByLocation': 0,
    'TimeSinceLastTxn': 0,
    'TxnCountInLast24Hours': 0,
    'RiskScore': 0,
    'IsAnomaly': 0
})

print("✅ Nuevas columnas creadas:")
print([col for col in df.columns if col in [
    'DayOfWeek','IsWeekend','IsLateNight','TimeOfDay',
    'AmountZScoreByLocation','TimeSinceLastTxn','TxnCountInLast24Hours','RiskScore','IsAnomaly'
]])
print("\nShape final:", df.shape)

[10/22/25 16:23:41] INFO     Loading data from PRI_full (ParquetDataset)...                    ]8;id=489318;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=479844;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[10/22/25 16:23:45] WARNING  C:\Users\AlexandreKz\AppData\Local\Temp\ipykernel_15268\2741150868.py: warnings.py:110
                             32: FutureWarning: The default of observed=False is deprecated and                    
                             will be changed to True in a future version of pandas. Pass                           
                             observed=False to retain current behavior or observed=True to adopt                   
                             the future default and silence this warning.                                          
                               df.groupby('CustLocation')['TransactionAmount (INR)']                               
                                                                                                                   

✅ Nuevas columnas creadas:
['DayOfWeek', 'IsWeekend', 'IsLateNight', 'TimeOfDay', 'AmountZScoreByLocation', 'TimeSinceLastTxn', 'TxnCountInLast24Hours', 'RiskScore', 'IsAnomaly']

Shape final: (984240, 31)


Ya con las variables agregadas necesitamos saber si son fraude, por lo que añadimos la etiqueta a partir de las transacciones existentes en el dataset fraud

In [6]:
#Crear etiqueta de fraude 1 Si esta en el fraud_Datset y 0 si no :p

df_full = catalog.load("PRI_full")
df_fraud = catalog.load("fraud_dataset")

# Etiqueta de fraude
df_full['is_fraud'] = df_full['TransactionID'].isin(df_fraud['TransactionID']).astype(int)

print(df_full['is_fraud'].value_counts())

[10/22/25 16:23:56] INFO     Loading data from PRI_full (ParquetDataset)...                    ]8;id=222815;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=219163;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

[10/22/25 16:23:57] INFO     Loading data from fraud_dataset (CSVDataset)...                   ]8;id=358467;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=461647;file://C:\Users\AlexandreKz\Documents\GitHub\Ml_Bank\venv\Lib\site-packages\kedro\io\data_catalog.py#1046\1046]8;;\

is_fraud
0    974397
1      9843
Name: count, dtype: int64


**//////////////////////////////////////////////////////////////////////////////////////////////////////////**
**//////////////////////////////////////////////////////////////////////////////////////////////////////////**
**//////////////////////////////////////////////////////////////////////////////////////////////////////////**
**//////////////////////////////////////////////////////////////////////////////////////////////////////////**
**//////////////////////////////////////////////////////////////////////////////////////////////////////////**